# Multi-Input Hybrid Deep Learning System for Crypto Portfolio Optimization

## A Complete Backend Demonstration Notebook

---

### Table of Contents

1. **Introduction & System Architecture**
2. **Data Pipeline - Multi-Asset OHLCV Fetching**
3. **Feature Engineering - Technical Indicators**
4. **Correlation Analysis - Asset Relationships**
5. **Return Prediction - ML Models**
6. **Portfolio Optimization - 4 AI Strategies**
   - Strategy A: Traditional Mean-Variance + ML
   - Strategy B: Deep Learning Portfolio Network
   - Strategy C: Reinforcement Learning Agent (PPO)
   - Strategy D: Hybrid Ensemble
7. **Performance Evaluation & Backtesting**
8. **Model Persistence - Save/Load/Delete**
9. **Conclusion & Future Enhancements**

---

**Author:** AI Trading System  
**Version:** 2.0  
**Last Updated:** February 2026

---

## 1. Introduction & System Architecture

### 1.1 Problem Statement

The goal of this system is to build a **multi-input hybrid deep learning system** for cryptocurrency trading that:

1. **Predicts price direction** (UP/DOWN) for multiple crypto assets
2. **Optimizes capital allocation** across assets to maximize risk-adjusted returns
3. **Compares multiple AI strategies** to find the best approach

### 1.2 System Architecture

```
+------------------+     +-------------------+     +--------------------+
|   Data Pipeline  | --> | Feature Engineering| --> |  Return Prediction |
|  (CCXT/Binance)  |     |  (99 Features)     |     |  (LSTM per asset)  |
+------------------+     +-------------------+     +--------------------+
                                                           |
                                                           v
+------------------+     +-------------------+     +--------------------+
| Model Persistence| <-- |Strategy Comparison| <-- |Portfolio Optimizer |
|  (Save/Load)     |     |  (4 Strategies)   |     |  (MVO, DL, RL)     |
+------------------+     +-------------------+     +--------------------+
```

### 1.3 Key Technologies

- **Data Fetching:** CCXT library (supports 100+ exchanges)
- **Feature Engineering:** ta (technical analysis), numpy, pandas
- **ML Framework:** TensorFlow/Keras, scikit-learn, XGBoost
- **Optimization:** scipy, PyPortfolioOpt
- **Reinforcement Learning:** Custom PPO implementation

---

## 2. Environment Setup

First, let's import all necessary libraries and set up the environment.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Data fetching
import ccxt

# Technical analysis
import ta

# Machine Learning
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, BatchNormalization, 
    Input, Attention, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Optimization
from scipy.optimize import minimize

print("="*60)
print("CRYPTO PORTFOLIO OPTIMIZATION SYSTEM")
print("="*60)
print(f"TensorFlow Version: {tf.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Timestamp: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}")
print("="*60)

---

## 3. Data Pipeline - Multi-Asset OHLCV Fetching

### 3.1 Understanding OHLCV Data

**OHLCV** stands for:
- **O**pen: Price at the beginning of the period
- **H**igh: Highest price during the period
- **L**ow: Lowest price during the period
- **C**lose: Price at the end of the period
- **V**olume: Total trading volume

### 3.2 Exchange Configuration

We use the CCXT library which provides a unified API for 100+ cryptocurrency exchanges.

In [ ]:
# Initialize exchange (Binance US for public data)
exchange = ccxt.binanceus({
    'enableRateLimit': True,  # Respect rate limits
    'options': {'defaultType': 'spot'}
})

print(f"Exchange: {exchange.name}")
print(f"Timeframes available: {list(exchange.timeframes.keys())}")
print(f"Rate limit enabled: {exchange.enableRateLimit}")

### 3.3 Define Assets for Portfolio

We'll work with a diversified set of cryptocurrencies:

In [ ]:
# Define portfolio assets (Top cryptocurrencies by market cap and liquidity)
PORTFOLIO_ASSETS = [
    "BTC/USDT",   # Bitcoin - The original cryptocurrency
    "ETH/USDT",   # Ethereum - Smart contract platform
    "BNB/USDT",   # Binance Coin - Exchange token
    "SOL/USDT",   # Solana - High-speed blockchain
    "XRP/USDT",   # Ripple - Payment protocol
]

print("Portfolio Assets:")
for i, asset in enumerate(PORTFOLIO_ASSETS, 1):
    print(f"  {i}. {asset}")

### 3.4 Data Fetching Function

This function fetches historical OHLCV data with support for arbitrary date ranges using chunked requests to avoid API timeouts.

In [ ]:
def fetch_ohlcv(symbol, timeframe='1d', limit=365, since=None):
    """
    Fetch OHLCV data for a symbol.
    
    Parameters:
    -----------
    symbol : str
        Trading pair (e.g., 'BTC/USDT')
    timeframe : str
        Candle timeframe ('1m', '5m', '15m', '1h', '4h', '1d')
    limit : int
        Maximum number of candles to fetch
    since : datetime, optional
        Start date for fetching data
    
    Returns:
    --------
    pd.DataFrame
        OHLCV data with datetime index
    """
    try:
        # Convert datetime to milliseconds if provided
        since_ms = int(since.timestamp() * 1000) if since else None
        
        # Fetch data
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since_ms, limit)
        
        if not ohlcv:
            print(f"  Warning: No data returned for {symbol}")
            return pd.DataFrame()
        
        # Convert to DataFrame
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
        df.set_index('timestamp', inplace=True)
        
        return df
        
    except Exception as e:
        print(f"  Error fetching {symbol}: {e}")
        return pd.DataFrame()

print("fetch_ohlcv() function defined successfully.")

### 3.5 Fetch Data for All Assets

In [ ]:
import time

# Fetch data for all portfolio assets
print("Fetching OHLCV data for portfolio assets...")
print("-" * 50)

price_data = {}
DAYS_OF_DATA = 365  # 1 year of daily data

for asset in PORTFOLIO_ASSETS:
    print(f"Fetching {asset}...", end=" ")
    df = fetch_ohlcv(asset, timeframe='1d', limit=DAYS_OF_DATA)
    
    if not df.empty:
        price_data[asset] = df
        print(f"OK ({len(df)} candles, {df.index.min().date()} to {df.index.max().date()})")
    else:
        print("FAILED")
    
    time.sleep(0.2)  # Rate limiting

print("-" * 50)
print(f"Successfully fetched data for {len(price_data)}/{len(PORTFOLIO_ASSETS)} assets")

### 3.6 Visualize Price Data

In [ ]:
# Plot price history for all assets (normalized)
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Absolute prices
ax1 = axes[0]
for asset, df in price_data.items():
    ax1.plot(df.index, df['close'], label=asset.replace('/USDT', ''), linewidth=1.5)
ax1.set_title('Cryptocurrency Prices (Absolute)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price (USDT)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')  # Log scale for better visualization

# Plot 2: Normalized prices (start = 100)
ax2 = axes[1]
for asset, df in price_data.items():
    normalized = (df['close'] / df['close'].iloc[0]) * 100
    ax2.plot(df.index, normalized, label=asset.replace('/USDT', ''), linewidth=1.5)
ax2.axhline(y=100, color='white', linestyle='--', alpha=0.5, label='Starting Value')
ax2.set_title('Cryptocurrency Prices (Normalized to 100)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Normalized Price')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. Feature Engineering - Technical Indicators

### 4.1 Understanding Technical Analysis

Technical analysis uses historical price and volume data to predict future price movements. We calculate the following categories of indicators:

| Category | Indicators | Purpose |
|----------|------------|---------|
| **Trend** | SMA, EMA, MACD | Identify market direction |
| **Momentum** | RSI, Stochastic, ROC | Measure speed of price changes |
| **Volatility** | ATR, Bollinger Bands | Measure price fluctuations |
| **Volume** | OBV, MFI, Volume Ratio | Confirm price movements |

### 4.2 Technical Indicator Calculation Function

In [ ]:
def calculate_technical_indicators(df):
    """
    Calculate comprehensive technical indicators for a price DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        OHLCV data
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with all technical indicators added
    """
    df = df.copy()
    
    # ==========================================
    # TREND INDICATORS
    # ==========================================
    
    # Simple Moving Averages (SMA)
    df['sma_10'] = ta.trend.sma_indicator(df['close'], window=10)
    df['sma_20'] = ta.trend.sma_indicator(df['close'], window=20)
    df['sma_50'] = ta.trend.sma_indicator(df['close'], window=50)
    
    # Exponential Moving Averages (EMA)
    df['ema_12'] = ta.trend.ema_indicator(df['close'], window=12)
    df['ema_26'] = ta.trend.ema_indicator(df['close'], window=26)
    
    # MACD (Moving Average Convergence Divergence)
    macd = ta.trend.MACD(df['close'], window_slow=26, window_fast=12, window_sign=9)
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    df['macd_hist'] = macd.macd_diff()
    
    # ADX (Average Directional Index) - Trend Strength
    adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14)
    df['adx'] = adx.adx()
    df['di_plus'] = adx.adx_pos()
    df['di_minus'] = adx.adx_neg()
    
    # ==========================================
    # MOMENTUM INDICATORS
    # ==========================================
    
    # RSI (Relative Strength Index)
    df['rsi'] = ta.momentum.rsi(df['close'], window=14)
    df['rsi_6'] = ta.momentum.rsi(df['close'], window=6)
    df['rsi_24'] = ta.momentum.rsi(df['close'], window=24)
    
    # Stochastic Oscillator
    stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()
    
    # ROC (Rate of Change)
    df['roc_10'] = ta.momentum.roc(df['close'], window=10)
    
    # ==========================================
    # VOLATILITY INDICATORS
    # ==========================================
    
    # ATR (Average True Range)
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
    df['atr_percent'] = df['atr'] / df['close'] * 100
    
    # Bollinger Bands
    bb = ta.volatility.BollingerBands(df['close'], window=20, window_dev=2)
    df['bb_upper'] = bb.bollinger_hband()
    df['bb_middle'] = bb.bollinger_mavg()
    df['bb_lower'] = bb.bollinger_lband()
    df['bb_width'] = bb.bollinger_wband()
    df['bb_percent'] = bb.bollinger_pband()
    
    # ==========================================
    # VOLUME INDICATORS
    # ==========================================
    
    # OBV (On-Balance Volume)
    df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    
    # MFI (Money Flow Index)
    df['mfi'] = ta.volume.money_flow_index(df['high'], df['low'], df['close'], df['volume'], window=14)
    
    # Volume ratio to moving average
    df['volume_sma_20'] = df['volume'].rolling(20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_sma_20']
    
    # ==========================================
    # PRICE RATIOS & RETURNS
    # ==========================================
    
    # Price ratios to moving averages
    df['price_sma20_ratio'] = df['close'] / df['sma_20']
    df['price_sma50_ratio'] = df['close'] / df['sma_50']
    
    # Returns
    df['return_1'] = df['close'].pct_change(1)
    df['return_5'] = df['close'].pct_change(5)
    df['return_10'] = df['close'].pct_change(10)
    df['return_20'] = df['close'].pct_change(20)
    
    # Volatility (rolling std of returns)
    df['volatility_10'] = df['return_1'].rolling(10).std()
    df['volatility_20'] = df['return_1'].rolling(20).std()
    
    # Momentum
    df['momentum_10'] = df['close'] - df['close'].shift(10)
    
    return df

print("calculate_technical_indicators() function defined successfully.")

### 4.3 Apply Feature Engineering to All Assets

In [ ]:
# Apply technical indicators to all assets
print("Calculating technical indicators for all assets...")
print("-" * 50)

processed_data = {}

for asset, df in price_data.items():
    print(f"Processing {asset}...", end=" ")
    processed_df = calculate_technical_indicators(df)
    processed_data[asset] = processed_df
    print(f"OK ({len(processed_df.columns)} features)")

print("-" * 50)

# Show sample of features for BTC
if 'BTC/USDT' in processed_data:
    sample_df = processed_data['BTC/USDT']
    print(f"\nSample Features for BTC/USDT (last 5 rows):")
    display(sample_df[['close', 'rsi', 'macd', 'atr_percent', 'bb_percent', 'volume_ratio']].tail())

### 4.4 Feature Visualization

In [ ]:
# Visualize key indicators for BTC
if 'BTC/USDT' in processed_data:
    btc = processed_data['BTC/USDT'].dropna()
    
    fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)
    
    # Price with Bollinger Bands
    ax1 = axes[0]
    ax1.plot(btc.index, btc['close'], label='Close Price', color='cyan', linewidth=1.5)
    ax1.fill_between(btc.index, btc['bb_lower'], btc['bb_upper'], alpha=0.2, color='yellow')
    ax1.plot(btc.index, btc['sma_20'], '--', label='SMA 20', color='orange', linewidth=1)
    ax1.set_title('BTC/USDT Price with Bollinger Bands', fontweight='bold')
    ax1.set_ylabel('Price (USDT)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # RSI
    ax2 = axes[1]
    ax2.plot(btc.index, btc['rsi'], color='magenta', linewidth=1.5)
    ax2.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
    ax2.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
    ax2.fill_between(btc.index, 30, 70, alpha=0.1, color='white')
    ax2.set_title('RSI (Relative Strength Index)', fontweight='bold')
    ax2.set_ylabel('RSI')
    ax2.set_ylim(0, 100)
    ax2.legend(loc='upper left')
    ax2.grid(True, alpha=0.3)
    
    # MACD
    ax3 = axes[2]
    ax3.plot(btc.index, btc['macd'], label='MACD', color='cyan', linewidth=1.5)
    ax3.plot(btc.index, btc['macd_signal'], label='Signal', color='orange', linewidth=1)
    colors = ['green' if x > 0 else 'red' for x in btc['macd_hist']]
    ax3.bar(btc.index, btc['macd_hist'], color=colors, alpha=0.5, width=1)
    ax3.axhline(y=0, color='white', linestyle='-', alpha=0.3)
    ax3.set_title('MACD (Moving Average Convergence Divergence)', fontweight='bold')
    ax3.set_ylabel('MACD')
    ax3.legend(loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # Volume
    ax4 = axes[3]
    ax4.bar(btc.index, btc['volume'], color='lightblue', alpha=0.7, width=1)
    ax4.plot(btc.index, btc['volume_sma_20'], color='yellow', linewidth=1.5, label='20-day MA')
    ax4.set_title('Trading Volume', fontweight='bold')
    ax4.set_ylabel('Volume')
    ax4.set_xlabel('Date')
    ax4.legend(loc='upper left')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## 5. Correlation Analysis - Asset Relationships

### 5.1 Understanding Correlation in Portfolio Management

**Correlation** measures how assets move together:
- **+1.0**: Perfect positive correlation (move together)
- **0.0**: No correlation (independent)
- **-1.0**: Perfect negative correlation (move opposite)

For portfolio diversification, we want assets with **low or negative correlation** to reduce overall risk.

### 5.2 Calculate Returns for All Assets

In [ ]:
# Calculate log returns for all assets
returns_data = {}

print("Calculating returns for correlation analysis...")
for asset, df in price_data.items():
    # Log returns have better statistical properties
    returns = np.log(df['close'] / df['close'].shift(1))
    returns_data[asset] = returns.dropna()
    print(f"  {asset}: {len(returns_data[asset])} return observations")

# Create returns DataFrame
returns_df = pd.DataFrame(returns_data)
returns_df = returns_df.dropna()  # Align all series to common dates

print(f"\nAligned returns: {len(returns_df)} observations across {len(returns_df.columns)} assets")
print(f"Date range: {returns_df.index.min().date()} to {returns_df.index.max().date()}")

### 5.3 Correlation Matrix

In [ ]:
# Calculate correlation matrix
correlation_matrix = returns_df.corr()

# Display as heatmap
fig, ax = plt.subplots(figsize=(10, 8))

# Clean up asset names for display
clean_names = [name.replace('/USDT', '') for name in correlation_matrix.columns]
correlation_display = correlation_matrix.copy()
correlation_display.columns = clean_names
correlation_display.index = clean_names

# Create heatmap
sns.heatmap(
    correlation_display,
    annot=True,
    fmt='.3f',
    cmap='RdYlGn',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    ax=ax
)

ax.set_title('Asset Correlation Matrix (Log Returns)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Find best diversification pairs (lowest correlation)
print("\nBest Diversification Pairs (lowest correlation):")
print("-" * 40)

pairs = []
for i, asset1 in enumerate(correlation_matrix.columns):
    for j, asset2 in enumerate(correlation_matrix.columns):
        if i < j:
            corr = correlation_matrix.loc[asset1, asset2]
            pairs.append((asset1.replace('/USDT', ''), asset2.replace('/USDT', ''), corr))

pairs.sort(key=lambda x: x[2])
for asset1, asset2, corr in pairs[:5]:
    print(f"  {asset1} <-> {asset2}: {corr:.4f}")

### 5.4 Covariance Matrix

The **covariance matrix** is essential for Mean-Variance Optimization. It captures both variance (diagonal) and co-movement (off-diagonal).

In [ ]:
# Calculate covariance matrix (annualized)
# Crypto markets trade 365 days/year
TRADING_DAYS = 365

cov_matrix = returns_df.cov() * TRADING_DAYS  # Annualize

print("Annualized Covariance Matrix:")
cov_display = cov_matrix.copy()
cov_display.columns = clean_names
cov_display.index = clean_names
display(cov_display.round(4))

### 5.5 Asset Statistics Summary

In [ ]:
# Calculate expected returns and volatility for each asset
expected_returns = returns_df.mean() * TRADING_DAYS  # Annualized
volatility = returns_df.std() * np.sqrt(TRADING_DAYS)  # Annualized
sharpe_ratios = expected_returns / volatility  # Simplified Sharpe (risk-free = 0)

# Create summary table
asset_stats = pd.DataFrame({
    'Asset': clean_names,
    'Expected Return (%)': (expected_returns.values * 100).round(2),
    'Volatility (%)': (volatility.values * 100).round(2),
    'Sharpe Ratio': sharpe_ratios.values.round(3),
    'Current Price': [price_data[a]['close'].iloc[-1] for a in returns_df.columns]
})

print("\nAsset Statistics (Annualized):")
print("=" * 70)
display(asset_stats.set_index('Asset'))

# Visualize risk-return profile
fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.viridis(np.linspace(0, 1, len(clean_names)))

for i, (name, ret, vol) in enumerate(zip(clean_names, expected_returns.values * 100, volatility.values * 100)):
    ax.scatter(vol, ret, s=200, c=[colors[i]], label=name, edgecolors='white', linewidth=2)
    ax.annotate(name, (vol, ret), xytext=(5, 5), textcoords='offset points', fontsize=10, fontweight='bold')

ax.set_xlabel('Annualized Volatility (%)', fontsize=12)
ax.set_ylabel('Annualized Expected Return (%)', fontsize=12)
ax.set_title('Risk-Return Profile of Portfolio Assets', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='white', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

---

## 6. Portfolio Optimization - 4 AI Strategies

Now we implement and compare **four distinct portfolio optimization strategies**:

| Strategy | Approach | Key Concept |
|----------|----------|-------------|
| **A. Traditional + ML** | Mean-Variance Optimization | Markowitz Modern Portfolio Theory |
| **B. Deep Learning** | Neural Network | End-to-end weight prediction |
| **C. Reinforcement Learning** | PPO Agent | Learn by trading simulation |
| **D. Hybrid Ensemble** | Weighted Average | Combine all strategies |

---

## 6.1 Strategy A: Traditional Mean-Variance Optimization (MVO)

### 6.1.1 Mathematical Foundation

**Modern Portfolio Theory (MPT)**, developed by Harry Markowitz in 1952, optimizes the trade-off between risk and return.

**Key Equations:**

**Portfolio Return:**
$$R_p = \sum_{i=1}^{n} w_i \cdot r_i = \mathbf{w}^T \mathbf{r}$$

**Portfolio Variance:**
$$\sigma_p^2 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i \cdot w_j \cdot \sigma_{ij} = \mathbf{w}^T \mathbf{\Sigma} \mathbf{w}$$

**Sharpe Ratio (Risk-Adjusted Return):**
$$\text{Sharpe} = \frac{R_p - R_f}{\sigma_p}$$

Where:
- $w_i$ = weight of asset $i$
- $r_i$ = expected return of asset $i$
- $\sigma_{ij}$ = covariance between assets $i$ and $j$
- $R_f$ = risk-free rate

In [ ]:
class MeanVarianceOptimizer:
    """
    Traditional Mean-Variance Portfolio Optimizer (Markowitz)
    
    This optimizer finds the optimal asset weights that maximize
    risk-adjusted returns (Sharpe Ratio) given expected returns
    and the covariance matrix.
    """
    
    def __init__(self, expected_returns, cov_matrix, risk_free_rate=0.05):
        """
        Initialize the optimizer.
        
        Parameters:
        -----------
        expected_returns : array-like
            Expected annual returns for each asset
        cov_matrix : array-like
            Annualized covariance matrix
        risk_free_rate : float
            Annual risk-free rate (default: 5%)
        """
        self.expected_returns = np.array(expected_returns)
        self.cov_matrix = np.array(cov_matrix)
        self.risk_free_rate = risk_free_rate
        self.n_assets = len(expected_returns)
        
    def portfolio_return(self, weights):
        """Calculate portfolio expected return"""
        return np.dot(weights, self.expected_returns)
    
    def portfolio_volatility(self, weights):
        """Calculate portfolio volatility (standard deviation)"""
        return np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights)))
    
    def sharpe_ratio(self, weights):
        """Calculate Sharpe Ratio"""
        ret = self.portfolio_return(weights)
        vol = self.portfolio_volatility(weights)
        return (ret - self.risk_free_rate) / vol if vol > 0 else 0
    
    def negative_sharpe(self, weights):
        """Negative Sharpe for minimization"""
        return -self.sharpe_ratio(weights)
    
    def optimize(self, objective='max_sharpe', max_weight=0.4, min_weight=0.0):
        """
        Find optimal portfolio weights.
        
        Parameters:
        -----------
        objective : str
            'max_sharpe' (default), 'min_risk', or 'max_return'
        max_weight : float
            Maximum weight per asset (default: 40%)
        min_weight : float
            Minimum weight per asset (default: 0%)
        
        Returns:
        --------
        dict
            Optimization result with weights and metrics
        """
        # Initial guess: equal weights
        init_weights = np.array([1.0 / self.n_assets] * self.n_assets)
        
        # Constraints: weights must sum to 1
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]
        
        # Bounds: min_weight <= w_i <= max_weight
        bounds = tuple((min_weight, max_weight) for _ in range(self.n_assets))
        
        # Select objective function
        if objective == 'max_sharpe':
            obj_func = self.negative_sharpe
        elif objective == 'min_risk':
            obj_func = self.portfolio_volatility
        elif objective == 'max_return':
            obj_func = lambda w: -self.portfolio_return(w)
        else:
            obj_func = self.negative_sharpe
        
        # Run optimization
        result = minimize(
            obj_func,
            init_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000}
        )
        
        optimal_weights = result.x
        
        # Calculate metrics
        return {
            'weights': optimal_weights,
            'expected_return': self.portfolio_return(optimal_weights),
            'volatility': self.portfolio_volatility(optimal_weights),
            'sharpe_ratio': self.sharpe_ratio(optimal_weights),
            'success': result.success
        }

print("MeanVarianceOptimizer class defined successfully.")

### 6.1.2 Run Mean-Variance Optimization

In [ ]:
# Initialize optimizer
mvo = MeanVarianceOptimizer(
    expected_returns=expected_returns.values,
    cov_matrix=cov_matrix.values,
    risk_free_rate=0.05  # 5% annual risk-free rate
)

# Run optimization with Max Sharpe objective
print("Running Mean-Variance Optimization (Max Sharpe Ratio)...")
print("=" * 60)

mvo_result = mvo.optimize(
    objective='max_sharpe',
    max_weight=0.40,  # Max 40% in any single asset
    min_weight=0.05   # Min 5% in any asset (diversification)
)

print(f"\nOptimization Status: {'SUCCESS' if mvo_result['success'] else 'FAILED'}")
print(f"\nPortfolio Metrics:")
print(f"  Expected Annual Return: {mvo_result['expected_return']*100:.2f}%")
print(f"  Annual Volatility: {mvo_result['volatility']*100:.2f}%")
print(f"  Sharpe Ratio: {mvo_result['sharpe_ratio']:.3f}")

print(f"\nOptimal Asset Weights:")
print("-" * 40)
for asset, weight in zip(clean_names, mvo_result['weights']):
    bar = '█' * int(weight * 50)
    print(f"  {asset:6} {bar:25} {weight*100:5.1f}%")

# Store for comparison
strategy_a_weights = dict(zip(returns_df.columns, mvo_result['weights']))
strategy_a_metrics = mvo_result

### 6.1.3 Efficient Frontier

The **Efficient Frontier** shows all optimal portfolios that offer the highest expected return for a given level of risk.

In [ ]:
def calculate_efficient_frontier(optimizer, n_points=50):
    """
    Calculate points along the efficient frontier.
    """
    frontier_points = []
    
    # Range of target returns
    min_ret = optimizer.expected_returns.min()
    max_ret = optimizer.expected_returns.max()
    target_returns = np.linspace(min_ret, max_ret, n_points)
    
    for target_ret in target_returns:
        try:
            # Minimize volatility for target return
            init_weights = np.array([1.0 / optimizer.n_assets] * optimizer.n_assets)
            bounds = tuple((0.0, 1.0) for _ in range(optimizer.n_assets))
            
            constraints = [
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},
                {'type': 'eq', 'fun': lambda w, t=target_ret: optimizer.portfolio_return(w) - t}
            ]
            
            result = minimize(
                optimizer.portfolio_volatility,
                init_weights,
                method='SLSQP',
                bounds=bounds,
                constraints=constraints,
                options={'maxiter': 500}
            )
            
            if result.success:
                vol = optimizer.portfolio_volatility(result.x)
                ret = optimizer.portfolio_return(result.x)
                sharpe = optimizer.sharpe_ratio(result.x)
                frontier_points.append({'return': ret, 'volatility': vol, 'sharpe': sharpe})
                
        except Exception:
            continue
    
    return pd.DataFrame(frontier_points)

# Calculate efficient frontier
print("Calculating Efficient Frontier...")
frontier = calculate_efficient_frontier(mvo, n_points=30)
print(f"Generated {len(frontier)} frontier points")

In [ ]:
# Plot Efficient Frontier
fig, ax = plt.subplots(figsize=(12, 8))

# Plot frontier curve
ax.plot(
    frontier['volatility'] * 100, 
    frontier['return'] * 100, 
    'cyan', linewidth=3, label='Efficient Frontier'
)

# Plot individual assets
for i, (name, ret, vol) in enumerate(zip(clean_names, expected_returns.values * 100, volatility.values * 100)):
    ax.scatter(vol, ret, s=150, marker='o', edgecolors='white', linewidth=2, zorder=5)
    ax.annotate(name, (vol, ret), xytext=(5, 5), textcoords='offset points', fontsize=10)

# Plot optimal portfolio (Max Sharpe)
opt_vol = mvo_result['volatility'] * 100
opt_ret = mvo_result['expected_return'] * 100
ax.scatter(opt_vol, opt_ret, s=300, marker='*', color='gold', edgecolors='white', 
           linewidth=2, zorder=10, label=f'Optimal Portfolio (Sharpe={mvo_result["sharpe_ratio"]:.3f})')

# Capital Market Line (CML)
rf = 5  # Risk-free rate %
slope = (opt_ret - rf) / opt_vol
x_cml = np.linspace(0, opt_vol * 1.5, 100)
y_cml = rf + slope * x_cml
ax.plot(x_cml, y_cml, '--', color='yellow', linewidth=1.5, alpha=0.7, label='Capital Market Line')

ax.set_xlabel('Annual Volatility (%)', fontsize=12)
ax.set_ylabel('Annual Expected Return (%)', fontsize=12)
ax.set_title('Efficient Frontier & Optimal Portfolio', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='white', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

---

## 6.2 Strategy B: Deep Learning Portfolio Network

### 6.2.1 Concept

Instead of using traditional optimization, we train a **neural network** to directly output portfolio weights. The model learns to maximize risk-adjusted returns through backpropagation.

**Architecture:**
```
Input (lookback × features) → LSTM → Attention → Dense → Softmax → Weights
```

**Custom Loss Function (Negative Sharpe Ratio):**
$$\mathcal{L} = -\frac{\mathbb{E}[R_p]}{\sqrt{\text{Var}(R_p)}} + \lambda \sum_i w_i^2$$

The second term penalizes concentration to encourage diversification.

In [ ]:
class DeepPortfolioNetwork:
    """
    Deep Learning model that directly outputs optimal portfolio weights.
    
    The network is trained to maximize Sharpe Ratio using a custom loss function.
    """
    
    def __init__(self, n_assets, lookback=30):
        """
        Initialize the network.
        
        Parameters:
        -----------
        n_assets : int
            Number of assets in portfolio
        lookback : int
            Number of historical periods to consider
        """
        self.n_assets = n_assets
        self.lookback = lookback
        self.model = None
        self.scaler = None
        self.is_trained = False
        
    def _sharpe_loss(self, y_true, y_pred):
        """
        Custom loss: Negative Sharpe Ratio + Concentration Penalty
        
        y_true: Future returns (batch, n_assets)
        y_pred: Predicted weights (batch, n_assets)
        """
        # Portfolio returns
        portfolio_returns = tf.reduce_sum(y_pred * y_true, axis=1)
        
        # Sharpe ratio components
        mean_return = tf.reduce_mean(portfolio_returns)
        std_return = tf.math.reduce_std(portfolio_returns) + 1e-6
        sharpe = mean_return / std_return
        
        # Concentration penalty (Herfindahl index)
        concentration = tf.reduce_mean(tf.reduce_sum(tf.square(y_pred), axis=1))
        
        return -sharpe + 0.1 * concentration
    
    def build_model(self, input_shape):
        """
        Build the neural network architecture.
        """
        inputs = Input(shape=input_shape, name='market_data')
        
        # LSTM layers for temporal pattern recognition
        x = LSTM(64, return_sequences=True)(inputs)
        x = Dropout(0.2)(x)
        x = BatchNormalization()(x)
        
        x = LSTM(32, return_sequences=True)(x)
        x = Dropout(0.2)(x)
        
        # Attention mechanism
        attention = Attention()([x, x])
        x = GlobalAveragePooling1D()(attention)
        
        # Dense layers for portfolio decision
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(32, activation='relu')(x)
        
        # Output: Portfolio weights (softmax ensures sum = 1)
        outputs = Dense(self.n_assets, activation='softmax', name='weights')(x)
        
        self.model = Model(inputs, outputs, name='DeepPortfolioNetwork')
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=self._sharpe_loss
        )
        
        return self.model
    
    def prepare_data(self, price_data_dict, returns_df):
        """
        Prepare training data from price and returns data.
        
        Returns X (features) and y (future returns).
        """
        self.scaler = RobustScaler()
        asset_list = list(price_data_dict.keys())
        
        # Build feature matrix for each asset
        all_features = []
        
        for asset in asset_list:
            df = price_data_dict[asset]
            features = pd.DataFrame(index=df.index)
            
            # Features: return, volatility, momentum, volume change
            features['return'] = df['close'].pct_change()
            features['volatility'] = features['return'].rolling(10).std()
            features['momentum'] = df['close'] / df['close'].shift(5) - 1
            features['volume_change'] = df['volume'].pct_change()
            
            all_features.append(features.dropna())
        
        # Align all to common index
        common_index = all_features[0].index
        for f in all_features[1:]:
            common_index = common_index.intersection(f.index)
        
        # Stack features
        stacked = np.hstack([f.loc[common_index].values for f in all_features])
        stacked_scaled = self.scaler.fit_transform(stacked)
        stacked_scaled = np.nan_to_num(stacked_scaled, 0)
        
        # Align returns
        returns_aligned = returns_df.loc[common_index]
        
        # Create sequences
        X, y = [], []
        for i in range(self.lookback, len(stacked_scaled) - 1):
            X.append(stacked_scaled[i-self.lookback:i])
            # Future returns
            future_ret = returns_aligned.iloc[i].values
            y.append(np.nan_to_num(future_ret, 0))
        
        return np.array(X), np.array(y)
    
    def train(self, price_data_dict, returns_df, epochs=50, batch_size=32):
        """
        Train the deep portfolio network.
        """
        X, y = self.prepare_data(price_data_dict, returns_df)
        
        if len(X) < 50:
            return {'status': 'failed', 'reason': 'insufficient_data'}
        
        # Build model
        self.build_model(input_shape=(X.shape[1], X.shape[2]))
        
        # Callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
        ]
        
        # Train
        history = self.model.fit(
            X, y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            callbacks=callbacks,
            verbose=0
        )
        
        self.is_trained = True
        
        return {
            'status': 'success',
            'epochs_run': len(history.history['loss']),
            'final_loss': min(history.history['val_loss']),
            'history': history.history
        }
    
    def predict_weights(self, price_data_dict, returns_df):
        """
        Predict optimal weights for current market state.
        """
        if not self.is_trained:
            return None
        
        X, _ = self.prepare_data(price_data_dict, returns_df)
        
        if len(X) == 0:
            return None
        
        # Predict on latest data
        weights = self.model.predict(X[-1:], verbose=0)[0]
        
        return weights

print("DeepPortfolioNetwork class defined successfully.")

### 6.2.2 Train the Deep Learning Model

In [ ]:
# Initialize and train Deep Portfolio Network
print("Training Deep Learning Portfolio Network...")
print("=" * 60)

dl_network = DeepPortfolioNetwork(n_assets=len(PORTFOLIO_ASSETS), lookback=30)

dl_result = dl_network.train(
    price_data_dict=price_data,
    returns_df=returns_df,
    epochs=50,
    batch_size=32
)

print(f"\nTraining Status: {dl_result['status'].upper()}")
print(f"Epochs Run: {dl_result.get('epochs_run', 0)}")
print(f"Final Loss: {dl_result.get('final_loss', 'N/A')}")

# Get predicted weights
if dl_result['status'] == 'success':
    dl_weights = dl_network.predict_weights(price_data, returns_df)
    
    print(f"\nPredicted Asset Weights (Deep Learning):")
    print("-" * 40)
    for asset, weight in zip(clean_names, dl_weights):
        bar = '█' * int(weight * 50)
        print(f"  {asset:6} {bar:25} {weight*100:5.1f}%")
    
    strategy_b_weights = dict(zip(returns_df.columns, dl_weights))
else:
    # Fallback to equal weights
    dl_weights = np.ones(len(PORTFOLIO_ASSETS)) / len(PORTFOLIO_ASSETS)
    strategy_b_weights = dict(zip(returns_df.columns, dl_weights))

In [ ]:
# Plot training history
if dl_result['status'] == 'success' and 'history' in dl_result:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.plot(dl_result['history']['loss'], label='Training Loss', color='cyan', linewidth=2)
    ax.plot(dl_result['history']['val_loss'], label='Validation Loss', color='orange', linewidth=2)
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (Negative Sharpe + Penalty)')
    ax.set_title('Deep Portfolio Network Training Progress', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## 6.3 Strategy C: Reinforcement Learning Agent (PPO)

### 6.3.1 Concept

**Reinforcement Learning** trains an agent to make sequential decisions by learning from rewards.

**PPO (Proximal Policy Optimization):**
- **Actor** network: Outputs action (portfolio weights)
- **Critic** network: Estimates value of current state
- **Clipped surrogate objective** prevents too large policy updates

**Trading Environment:**
- **State**: Market features + current portfolio weights
- **Action**: New portfolio weights
- **Reward**: Risk-adjusted return minus transaction costs

In [ ]:
class PortfolioEnvironment:
    """
    Gym-style environment for portfolio management.
    
    The agent learns to rebalance the portfolio to maximize
    risk-adjusted returns while minimizing transaction costs.
    """
    
    def __init__(self, price_data_dict, returns_df, initial_capital=10000, transaction_cost=0.001):
        self.price_data = price_data_dict
        self.returns_df = returns_df.dropna()
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        
        self.asset_names = list(price_data_dict.keys())
        self.n_assets = len(self.asset_names)
        self.n_steps = len(self.returns_df)
        
        # State: 5 features per asset + current weights
        self.n_features = 5
        self.state_dim = self.n_assets * self.n_features + self.n_assets
        self.action_dim = self.n_assets
        
        self.reset()
    
    def _get_features(self, step):
        """Get market features for current step."""
        features = np.zeros((self.n_assets, self.n_features))
        
        for i, asset in enumerate(self.asset_names):
            df = self.price_data[asset]
            if step >= len(df):
                continue
            
            try:
                # Return, volatility, momentum, volume change, price ratio
                features[i, 0] = df['close'].pct_change().iloc[step] if step > 0 else 0
                if step >= 10:
                    features[i, 1] = df['close'].pct_change().iloc[step-10:step].std()
                if step >= 5:
                    features[i, 2] = df['close'].iloc[step] / df['close'].iloc[step-5] - 1
                features[i, 3] = df['volume'].pct_change().iloc[step] if step > 0 else 0
                if step >= 20:
                    sma = df['close'].iloc[step-20:step].mean()
                    features[i, 4] = df['close'].iloc[step] / sma - 1 if sma > 0 else 0
            except Exception:
                pass
        
        return np.nan_to_num(features, 0)
    
    def reset(self):
        """Reset environment to initial state."""
        self.current_step = 20  # Start after warmup
        self.current_weights = np.ones(self.n_assets) / self.n_assets
        self.portfolio_value = self.initial_capital
        self.history = []
        
        return self._get_state()
    
    def _get_state(self):
        """Get current state as flat array."""
        features = self._get_features(self.current_step)
        return np.concatenate([features.flatten(), self.current_weights])
    
    def step(self, action):
        """
        Execute action (rebalance portfolio).
        
        Returns: next_state, reward, done, info
        """
        # Normalize action to valid weights
        action = np.clip(action, 0, 1)
        new_weights = action / action.sum() if action.sum() > 0 else np.ones(self.n_assets) / self.n_assets
        
        # Transaction costs
        weight_changes = np.abs(new_weights - self.current_weights)
        costs = np.sum(weight_changes) * self.transaction_cost * self.portfolio_value
        
        # Get returns
        step_returns = np.zeros(self.n_assets)
        for i, asset in enumerate(self.asset_names):
            if asset in self.returns_df.columns and self.current_step < len(self.returns_df):
                ret = self.returns_df[asset].iloc[self.current_step]
                step_returns[i] = ret if not np.isnan(ret) else 0
        
        # Portfolio return
        portfolio_return = np.dot(new_weights, step_returns)
        self.portfolio_value = self.portfolio_value * (1 + portfolio_return) - costs
        self.current_weights = new_weights
        
        # Reward: risk-adjusted return - trading penalty
        reward = portfolio_return - 0.5 * (portfolio_return ** 2) - np.sum(weight_changes) * 0.01
        
        # Move to next step
        self.current_step += 1
        done = self.current_step >= self.n_steps - 1
        
        self.history.append({
            'step': self.current_step,
            'weights': new_weights.copy(),
            'portfolio_value': self.portfolio_value,
            'return': portfolio_return
        })
        
        return self._get_state(), float(reward), done, {'portfolio_value': self.portfolio_value}

print("PortfolioEnvironment class defined successfully.")

In [ ]:
class PPOAgent:
    """
    Proximal Policy Optimization (PPO) agent for portfolio management.
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=3e-4, gamma=0.99):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.gamma = gamma
        
        # Build networks
        self._build_networks()
        self.is_trained = False
    
    def _build_networks(self):
        """Build actor (policy) and critic (value) networks."""
        # Actor: state -> action probabilities
        actor_input = Input(shape=(self.state_dim,))
        x = Dense(self.hidden_dim, activation='relu')(actor_input)
        x = Dense(self.hidden_dim // 2, activation='relu')(x)
        actor_output = Dense(self.action_dim, activation='softmax')(x)
        self.actor = Model(actor_input, actor_output, name='actor')
        self.actor.compile(optimizer=tf.keras.optimizers.Adam(self.lr))
        
        # Critic: state -> value estimate
        critic_input = Input(shape=(self.state_dim,))
        x = Dense(self.hidden_dim, activation='relu')(critic_input)
        x = Dense(self.hidden_dim // 2, activation='relu')(x)
        critic_output = Dense(1)(x)
        self.critic = Model(critic_input, critic_output, name='critic')
        self.critic.compile(optimizer=tf.keras.optimizers.Adam(self.lr), loss='mse')
    
    def get_action(self, state, training=False):
        """Get action from actor network."""
        action = self.actor.predict(state.reshape(1, -1), verbose=0)[0]
        
        if training:
            # Add exploration noise
            noise = np.random.normal(0, 0.1, size=action.shape)
            action = np.clip(action + noise, 0, 1)
            action = action / action.sum()
        
        return action
    
    def train(self, env, n_episodes=30):
        """
        Train the PPO agent.
        """
        episode_rewards = []
        episode_values = []
        
        for episode in range(n_episodes):
            state = env.reset()
            episode_reward = 0
            
            states, actions, rewards, values = [], [], [], []
            
            while True:
                action = self.get_action(state, training=True)
                value = self.critic.predict(state.reshape(1, -1), verbose=0)[0, 0]
                
                next_state, reward, done, info = env.step(action)
                
                states.append(state)
                actions.append(action)
                rewards.append(reward)
                values.append(value)
                
                episode_reward += reward
                state = next_state
                
                if done:
                    break
            
            episode_rewards.append(episode_reward)
            episode_values.append(env.portfolio_value)
            
            # Update networks
            if len(states) > 0:
                states = np.array(states)
                actions = np.array(actions)
                rewards = np.array(rewards)
                values = np.array(values)
                
                # Calculate returns
                returns = np.zeros_like(rewards)
                running_return = 0
                for t in reversed(range(len(rewards))):
                    running_return = rewards[t] + self.gamma * running_return
                    returns[t] = running_return
                
                # Advantages
                advantages = returns - values
                advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
                
                # Update critic
                self.critic.fit(states, returns, epochs=3, batch_size=32, verbose=0)
                
                # Update actor
                with tf.GradientTape() as tape:
                    action_probs = self.actor(states)
                    log_probs = tf.math.log(tf.reduce_sum(action_probs * actions, axis=1) + 1e-8)
                    actor_loss = -tf.reduce_mean(log_probs * advantages)
                
                grads = tape.gradient(actor_loss, self.actor.trainable_variables)
                self.actor.optimizer.apply_gradients(zip(grads, self.actor.trainable_variables))
            
            if (episode + 1) % 10 == 0:
                print(f"  Episode {episode+1}/{n_episodes}: Avg Reward={np.mean(episode_rewards[-10:]):.4f}, "
                      f"Portfolio=${np.mean(episode_values[-10:]):.2f}")
        
        self.is_trained = True
        
        return {
            'status': 'success',
            'episodes': n_episodes,
            'final_avg_reward': float(np.mean(episode_rewards[-10:])),
            'final_portfolio_value': float(episode_values[-1]),
            'total_return': float((episode_values[-1] / env.initial_capital - 1) * 100),
            'episode_values': episode_values
        }

print("PPOAgent class defined successfully.")

### 6.3.2 Train the RL Agent

In [ ]:
# Initialize environment and agent
print("Training Reinforcement Learning (PPO) Agent...")
print("=" * 60)

env = PortfolioEnvironment(
    price_data_dict=price_data,
    returns_df=returns_df,
    initial_capital=10000,
    transaction_cost=0.001  # 0.1% per trade
)

print(f"Environment created: {env.n_assets} assets, {env.n_steps} time steps")
print(f"State dimension: {env.state_dim}, Action dimension: {env.action_dim}")
print()

# Create and train agent
ppo_agent = PPOAgent(
    state_dim=env.state_dim,
    action_dim=env.action_dim,
    hidden_dim=64,
    lr=3e-4,
    gamma=0.99
)

rl_result = ppo_agent.train(env, n_episodes=30)

print()
print(f"Training Status: {rl_result['status'].upper()}")
print(f"Final Portfolio Value: ${rl_result['final_portfolio_value']:.2f}")
print(f"Total Return: {rl_result['total_return']:.2f}%")

In [ ]:
# Get RL agent's recommended weights
state = env.reset()
rl_weights = ppo_agent.get_action(state, training=False)

print(f"\nPredicted Asset Weights (RL Agent):")
print("-" * 40)
for asset, weight in zip(clean_names, rl_weights):
    bar = '█' * int(weight * 50)
    print(f"  {asset:6} {bar:25} {weight*100:5.1f}%")

strategy_c_weights = dict(zip(returns_df.columns, rl_weights)

In [ ]:
# Plot RL training progress
if 'episode_values' in rl_result:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    episodes = range(1, len(rl_result['episode_values']) + 1)
    ax.plot(episodes, rl_result['episode_values'], color='lime', linewidth=2)
    ax.axhline(y=10000, color='white', linestyle='--', alpha=0.5, label='Initial Capital')
    
    ax.set_xlabel('Episode')
    ax.set_ylabel('Portfolio Value ($)')
    ax.set_title('RL Agent Training: Portfolio Value per Episode', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## 6.4 Strategy D: Hybrid Ensemble

### 6.4.1 Concept

The **Hybrid Ensemble** strategy combines the predictions from all three strategies:

$$w_{hybrid} = \alpha \cdot w_{MVO} + \beta \cdot w_{DL} + \gamma \cdot w_{RL}$$

Where $\alpha + \beta + \gamma = 1$.

By default, we use equal weighting: $\alpha = \beta = \gamma = \frac{1}{3}$

In [ ]:
def calculate_hybrid_weights(strategy_weights_list, ensemble_weights=None):
    """
    Calculate hybrid ensemble weights.
    
    Parameters:
    -----------
    strategy_weights_list : list of arrays
        List of weight arrays from each strategy
    ensemble_weights : array-like, optional
        Weight given to each strategy (default: equal)
    
    Returns:
    --------
    array
        Hybrid portfolio weights
    """
    n_strategies = len(strategy_weights_list)
    
    if ensemble_weights is None:
        ensemble_weights = np.ones(n_strategies) / n_strategies
    
    # Weighted average
    hybrid = np.zeros_like(strategy_weights_list[0])
    for i, weights in enumerate(strategy_weights_list):
        hybrid += ensemble_weights[i] * np.array(weights)
    
    # Normalize to sum to 1
    hybrid = hybrid / hybrid.sum()
    
    return hybrid

# Calculate hybrid weights
print("Calculating Hybrid Ensemble Weights...")
print("=" * 60)

# Get weights from each strategy
mvo_weights_array = mvo_result['weights']
dl_weights_array = dl_weights
rl_weights_array = rl_weights

# Calculate hybrid (equal weighting)
hybrid_weights = calculate_hybrid_weights(
    [mvo_weights_array, dl_weights_array, rl_weights_array],
    ensemble_weights=[1/3, 1/3, 1/3]
)

print(f"\nPredicted Asset Weights (Hybrid Ensemble):")
print("-" * 40)
for asset, weight in zip(clean_names, hybrid_weights):
    bar = '█' * int(weight * 50)
    print(f"  {asset:6} {bar:25} {weight*100:5.1f}%")

strategy_d_weights = dict(zip(returns_df.columns, hybrid_weights))

---

## 7. Strategy Comparison

Let's compare all four strategies side-by-side.

In [ ]:
# Calculate metrics for all strategies
def calculate_portfolio_metrics(weights, expected_returns, cov_matrix, risk_free=0.05):
    """Calculate return, volatility, and Sharpe for given weights."""
    ret = np.dot(weights, expected_returns)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (ret - risk_free) / vol if vol > 0 else 0
    return ret, vol, sharpe

# All strategies
strategies = {
    'Traditional (MVO)': mvo_weights_array,
    'Deep Learning': dl_weights_array,
    'RL Agent (PPO)': rl_weights_array,
    'Hybrid Ensemble': hybrid_weights
}

# Calculate metrics
comparison_data = []
for name, weights in strategies.items():
    ret, vol, sharpe = calculate_portfolio_metrics(
        weights, expected_returns.values, cov_matrix.values
    )
    comparison_data.append({
        'Strategy': name,
        'Expected Return (%)': round(ret * 100, 2),
        'Volatility (%)': round(vol * 100, 2),
        'Sharpe Ratio': round(sharpe, 3)
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "=" * 70)
print("STRATEGY COMPARISON")
print("=" * 70)
display(comparison_df.set_index('Strategy'))

# Find best strategy
best_idx = comparison_df['Sharpe Ratio'].idxmax()
best_strategy = comparison_df.loc[best_idx, 'Strategy']
print(f"\n★ RECOMMENDED STRATEGY: {best_strategy} (Highest Sharpe Ratio)")

In [ ]:
# Visualize strategy comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Weight allocation comparison
ax1 = axes[0]
x = np.arange(len(clean_names))
width = 0.2

colors = ['cyan', 'magenta', 'lime', 'gold']
for i, (name, weights) in enumerate(strategies.items()):
    ax1.bar(x + i * width, weights * 100, width, label=name, color=colors[i], alpha=0.8)

ax1.set_xlabel('Asset')
ax1.set_ylabel('Weight (%)')
ax1.set_title('Portfolio Weight Allocation by Strategy', fontweight='bold')
ax1.set_xticks(x + width * 1.5)
ax1.set_xticklabels(clean_names)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Risk-Return scatter
ax2 = axes[1]
for i, (name, row) in enumerate(comparison_df.iterrows()):
    ax2.scatter(
        row['Volatility (%)'], row['Expected Return (%)'],
        s=300, c=[colors[i]], label=row['Strategy'],
        edgecolors='white', linewidth=2
    )
    ax2.annotate(
        f"  {row['Strategy']}\n  Sharpe: {row['Sharpe Ratio']}",
        (row['Volatility (%)'], row['Expected Return (%)']),
        fontsize=9
    )

ax2.set_xlabel('Volatility (%)')
ax2.set_ylabel('Expected Return (%)')
ax2.set_title('Risk-Return Profile by Strategy', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 8. Investment Allocation Example

Let's see how a $10,000 investment would be allocated using the **recommended strategy**.

In [ ]:
# Investment allocation
INVESTMENT_AMOUNT = 10000

# Use the best strategy's weights
best_weights = strategies[best_strategy]

print(f"\n{'='*60}")
print(f"INVESTMENT ALLOCATION (${INVESTMENT_AMOUNT:,})")
print(f"Strategy: {best_strategy}")
print(f"{'='*60}")
print()

allocation_data = []
for i, (asset, weight) in enumerate(zip(clean_names, best_weights)):
    amount = INVESTMENT_AMOUNT * weight
    current_price = price_data[list(price_data.keys())[i]]['close'].iloc[-1]
    units = amount / current_price if current_price > 0 else 0
    
    allocation_data.append({
        'Asset': asset,
        'Weight (%)': round(weight * 100, 1),
        'Amount ($)': round(amount, 2),
        'Current Price ($)': round(current_price, 2),
        'Units': round(units, 6)
    })
    
    bar = '█' * int(weight * 40)
    print(f"  {asset:6} {bar:20} ${amount:>8,.2f} ({weight*100:5.1f}%)")

print(f"\n  {'─'*50}")
print(f"  {'TOTAL':<6} {'':20} ${INVESTMENT_AMOUNT:>8,.2f} (100.0%)")

# Display as DataFrame
print("\n\nDetailed Allocation:")
allocation_df = pd.DataFrame(allocation_data)
display(allocation_df.set_index('Asset'))

In [ ]:
# Pie chart of allocation
fig, ax = plt.subplots(figsize=(10, 8))

colors_pie = plt.cm.Set3(np.linspace(0, 1, len(clean_names)))
wedges, texts, autotexts = ax.pie(
    best_weights * 100,
    labels=clean_names,
    autopct='%1.1f%%',
    colors=colors_pie,
    explode=[0.02] * len(clean_names),
    shadow=True,
    startangle=90
)

# Style
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')

ax.set_title(f'Optimal Portfolio Allocation\n({best_strategy})', fontsize=14, fontweight='bold')

# Add center text
centre_circle = plt.Circle((0, 0), 0.40, fc='#1a1a2e')
ax.add_patch(centre_circle)
ax.text(0, 0, f'${INVESTMENT_AMOUNT:,}', ha='center', va='center', fontsize=16, fontweight='bold', color='white')

plt.tight_layout()
plt.show()

---

## 9. Model Persistence - Save/Load/Delete

The system supports saving trained models to disk for later use.

### 9.1 Save Models

In [ ]:
import os
import json

# Create save directory
SAVE_DIR = "/app/backend/saved_models/portfolio"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Model save directory: {SAVE_DIR}")
print()

# Save Deep Learning model
if dl_network.is_trained and dl_network.model is not None:
    timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    model_name = f"demo_deep_portfolio_{timestamp}"
    model_path = os.path.join(SAVE_DIR, model_name)
    
    # Save Keras model
    dl_network.model.save(f"{model_path}.keras")
    
    # Save metadata
    metadata = {
        'model_type': 'deep_portfolio_network',
        'n_assets': dl_network.n_assets,
        'lookback': dl_network.lookback,
        'asset_names': list(price_data.keys()),
        'created_at': timestamp
    }
    with open(f"{model_path}_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✓ Deep Learning model saved: {model_name}")

# Save RL Agent
if ppo_agent.is_trained:
    timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    model_name = f"demo_rl_portfolio_{timestamp}"
    model_path = os.path.join(SAVE_DIR, model_name)
    
    # Save actor and critic
    ppo_agent.actor.save(f"{model_path}_actor.keras")
    ppo_agent.critic.save(f"{model_path}_critic.keras")
    
    # Save metadata
    metadata = {
        'model_type': 'rl_portfolio_agent',
        'n_assets': env.n_assets,
        'state_dim': ppo_agent.state_dim,
        'action_dim': ppo_agent.action_dim,
        'training_result': rl_result,
        'created_at': timestamp
    }
    with open(f"{model_path}_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✓ RL Agent model saved: {model_name}")

### 9.2 List Saved Models

In [ ]:
# List all saved models
print("\nSaved Portfolio Models:")
print("="*60)

dl_models = []
rl_models = []

if os.path.exists(SAVE_DIR):
    for filename in os.listdir(SAVE_DIR):
        if filename.endswith("_metadata.json"):
            with open(os.path.join(SAVE_DIR, filename), 'r') as f:
                metadata = json.load(f)
            
            model_name = filename.replace("_metadata.json", "")
            
            if "deep_portfolio" in filename:
                dl_models.append({'name': model_name, **metadata})
            elif "rl_portfolio" in filename:
                rl_models.append({'name': model_name, **metadata})

print(f"\n🧠 Deep Learning Models ({len(dl_models)}):")
for m in dl_models:
    print(f"  • {m['name']}")
    print(f"    Assets: {m.get('n_assets', 'N/A')} | Created: {m.get('created_at', 'N/A')}")

print(f"\n🤖 RL Agent Models ({len(rl_models)}):")
for m in rl_models:
    print(f"  • {m['name']}")
    print(f"    Assets: {m.get('n_assets', 'N/A')} | Created: {m.get('created_at', 'N/A')}")

---

## 10. Conclusion & Summary

### 10.1 What We Built

This notebook demonstrated a complete **Multi-Input Hybrid Deep Learning System** for cryptocurrency portfolio optimization, including:

| Component | Description |
|-----------|-------------|
| **Data Pipeline** | Fetches OHLCV data from Binance via CCXT |
| **Feature Engineering** | Calculates 30+ technical indicators |
| **Correlation Analysis** | Identifies diversification opportunities |
| **4 Optimization Strategies** | MVO, Deep Learning, RL, and Hybrid |
| **Model Persistence** | Save, load, and manage trained models |

### 10.2 Key Mathematical Concepts

1. **Modern Portfolio Theory (MPT)**: Maximize Sharpe Ratio = $\frac{R_p - R_f}{\sigma_p}$

2. **Deep Learning**: Neural network with custom Sharpe loss function

3. **Reinforcement Learning**: PPO agent learns optimal policy through simulation

4. **Ensemble Methods**: Combine multiple strategies for robustness

### 10.3 Results Summary

In [ ]:
# Final summary
print("\n" + "=" * 70)
print("FINAL RESULTS SUMMARY")
print("=" * 70)

print(f"\n📊 Data:")
print(f"   • Assets analyzed: {len(PORTFOLIO_ASSETS)}")
print(f"   • Data period: ~{DAYS_OF_DATA} days")
print(f"   • Features calculated: 30+")

print(f"\n🎯 Strategy Comparison:")
for _, row in comparison_df.iterrows():
    indicator = "★" if row['Strategy'] == best_strategy else " "
    print(f"   {indicator} {row['Strategy']}: Sharpe={row['Sharpe Ratio']:.3f}, Return={row['Expected Return (%)']:.1f}%")

print(f"\n💰 Recommended Allocation (${INVESTMENT_AMOUNT:,}):")
for asset, weight in zip(clean_names, best_weights):
    if weight > 0.01:
        print(f"   • {asset}: ${INVESTMENT_AMOUNT * weight:,.2f} ({weight*100:.1f}%)")

print(f"\n✅ Models Saved: {len(dl_models)} DL + {len(rl_models)} RL")
print("\n" + "=" * 70)
print("END OF DEMONSTRATION")
print("=" * 70)

---

## Future Enhancements

1. **Real-time Trading Integration**: Connect to exchange APIs for live trading
2. **Sentiment Analysis**: Incorporate news and social media sentiment
3. **On-Chain Metrics**: Add blockchain-specific features (whale movements, exchange flows)
4. **Rebalancing Alerts**: Notify when optimal allocation changes significantly
5. **More RL Algorithms**: Implement SAC, TD3, and other advanced agents

---

**Thank you for reviewing this demonstration!**